## 先用executor.cores=16 启动
## 到后面再restart 用executor.cores=4 启动一次

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import datetime
from pyspark.sql import Window
from pyspark.sql.types import *
import sys
import os
app_name = 'selection bu batch100'
# spark = SparkSession.builder.appName(app_name).enableHiveSupport().getOrCreate()
os.environ['PYSPARK_PYTHON'] = "/usr/bin/python3"
os.environ['PYSPARK_DRIVER_PYTHON'] = "/usr/bin/python3"

spark = (SparkSession\
         .builder\
         .appName("test")\
         .enableHiveSupport()\
         .config("spark.executor.instances", "100")\
         .config("spark.executor.memory","48g")\
         .config("spark.executor.cores","4")\
         .config("spark.driver.memory","40g")\
         .config("spark.sql.shuffle.partitions","2000")\
         .config("spark.default.parallelism","2000")\
         .config("spark.driver.maxResultSize", "8g")\
         .config("spark.pyspark.python", "/usr/bin/python3")\
         .config("spark.sql.autoBroadcastJoinThreshold","-1")
         .config("spark.yarn.appMasterEnv.yarn.nodemanager.container-executor.class","DockerLinuxContainer")\
         .config("spark.executorEnv.yarn.nodemanager.container-executor.class","DockerLinuxContainer")\
         .config("spark.yarn.appMasterEnv.yarn.nodemanager.docker-container-executor.image-name","bdp-docker.jd.com:5000/wise_mart_rmb_py36:latest")\
         .config("spark.executorEnv.yarn.nodemanager.docker-container-executor.image-name","bdp-docker.jd.com:5000/wise_mart_rmb_py36:latest")\
         .getOrCreate())

spark.conf.set("hive.exec.dynamic.partition", "true")
spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict")

import datetime
import sys
import pandas as pd
import os
import numpy as npo
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
import pyspark.sql.types as sql_type
import spa_utils

spark.sql("set hive.exec.dynamic.partition=true")
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.sql('''set hive.exec.max.dynamic.partitions=200551''')
spark.sql('''set hive.exec.max.dynamic.partitions.pernode=200551''')
params = {'author':'xiaoxiao10'}

## 取特征

In [2]:
import datetime
import sys
import pandas as pd
import os
import numpy as np
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,f1_score,precision_score,recall_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.spatial.distance import cosine
name = locals()

In [3]:
from SPA_simulation_functions import *

df = spark.sql('''select * from dev.dev_xianzhi_model_data_xiaoxiao_6''')
# df = df.withColumn('target_price',F.log(F.col('target_price')))
df.cache()
df.count()

1043972

In [4]:
spark.sparkContext.addPyFile('SPA_simulation_functions.py')

selected_columns = ['item_sku_id', 'sale_qtty','cid3' , 'decomposedtrend','cid3trend','rolling360mean', 'rolling180mean',
                    'rolling90mean','rolling28mean','rolling14mean','rolling7mean','rolling5mean',
                    'rolling3mean','rolling2mean','rolling1mean','rolling14median','rolling7median',
                    'rolling360decaymean','rolling180decaymean','rolling90decaymean','rolling28decaymean',
                    'rolling14decaymean','rolling7decaymean','rolling3decaymean','uv','uv_1','uv_3','uv_7',
                    'uv_15','uv_30','stock_status','stock_qtty','stock_qtty_1','stock_status_1','stock_qtty_3',
                    'stock_status_3','stock_qtty_7','stock_status_7','stock_qtty_15','stock_status_15',
                    'stock_qtty_30','stock_status_30','redprice','redprice_1','redprice_3','redprice_7',
                    'redprice_15','redprice_30','dealprice','baseprice','base_qtty','baseprice_deal_1',
                    'dealprice_1','baseprice_deal_3','dealprice_3','baseprice_deal_7','dealprice_7',
                    'baseprice_deal_15','dealprice_15','baseprice_deal_30','dealprice_30','baseprice_red',
                    'days_flag', 'netprice','target_price','target_trend','target_cid3trend','target_days_flag',
                    'test_flag','target_qtty','valid_flag', 'dt']
X_SCHEMA_SKU = ['target_price', 'decomposedtrend','cid3trend','target_days_flag']
rolling_columns = []
SCHEMA_OUTPUT_SKU = StructType([
    StructField("dt", sql_type.StringType()),
    StructField("item_sku_id", sql_type.StringType()),
    StructField("target_price", sql_type.FloatType()),
    StructField("prediction", sql_type.FloatType()),
    StructField("r2_predict", sql_type.FloatType()),
    StructField("r2_test", sql_type.FloatType()),
    StructField("mse", sql_type.FloatType()),
    StructField("mape", sql_type.FloatType()),
    StructField("model_type", sql_type.StringType()),
    StructField("model", sql_type.StringType()),
    StructField("feature_importance", sql_type.StringType()),
    StructField("valid_flag", sql_type.FloatType()),
    StructField("target_qtty", sql_type.FloatType())
])


def format_result_sku(row):
    return (
        str(row['dt']),
        str(row['item_sku_id']),
        float(row['target_price']),
        float(row['prediction']),
        float(row['r2_predict']),
        float(row['r2_test']),
        float(row['mse']),
        float(row['mape']),
        str(row['model_type']),
        str(row['model']),
        str(row['feature_importance']),
        float(row['valid_flag']),
        float(row['target_qtty'])
    )


In [5]:
# update_predict = ['lr', 'hr', 'rf', 'prophet']
# 使用4种模型
update_predict = ['lr', 'rf', 'prophet','xgboost']

# 按照dt打标，valid_flag test_flag
df_dt = df.select('item_sku_id','dt').distinct()
df_dt = df_dt.withColumn('rank',F.row_number().over(Window.partitionBy('item_sku_id').orderBy(F.col('dt').desc())))\
             .withColumn('valid_flag',F.when(F.col('rank')<=45,1)\
                                       .when((F.col('rank')<=55)&(F.col('rank')>=46),2)\
                                       .when((F.col('rank')<=85)&(F.col('rank')>=56),3)\
                                       .when((F.col('rank')<=95)&(F.col('rank')>=86),4)\
                                       .when((F.col('rank')<=125)&(F.col('rank')>=96),5).otherwise(F.lit(0)))

df_lu = df.join(df_dt.select('item_sku_id','dt','valid_flag'),['item_sku_id','dt'],'inner')

# df_sku_count = df_lu.filter(F.col('valid_flag').isin([0,4,5])).groupBy('item_sku_id')\
df_sku_count = df_lu.filter(F.col('valid_flag').isin([0])).groupBy('item_sku_id')\
                    .agg(F.count('dt').alias('count'))\
                    .filter(F.col('count')>=30)

df_2 = df_lu.join(df_sku_count.select('item_sku_id'), ['item_sku_id'], 'inner')
df_2.cache()
df_2.count()

992492

In [6]:
# 输出验证集结果
# 每个sku有多个模型结果
# df_2 = df_2.withColumn('test_flag',F.when(F.col('valid_flag').isin([0,4,5]),F.lit(0)).otherwise(F.lit(1)))
df_2 = df_2.withColumn('test_flag',F.when(F.col('valid_flag').isin([0]),F.lit(0)).otherwise(F.lit(1)))
result2 = df_2.select(selected_columns).rdd.map(lambda row: ((row['item_sku_id']), row)).groupByKey()\
    .flatMap(lambda row : calculate_baseline_sku(row, update_predict, selected_columns, 
                                                 X_SCHEMA_SKU, rolling_columns, 'self'))

result_df2 = spark.createDataFrame(result2.map(format_result_sku), schema=SCHEMA_OUTPUT_SKU)

result_df2 = result_df2.na.drop(subset=['prediction'])
result_df2.cache()
result_df2.count()

856040

In [7]:
# 选模型
result_df2_pd = result_df2.drop('model','feature_importance').toPandas()
# result_df3_pd = result_df2_pd.loc[result_df2_pd['valid_flag']==2]\
result_df3_pd = result_df2_pd.loc[result_df2_pd['valid_flag']==4]\
                             .groupby(['item_sku_id','model_type'])\
                             .apply(lambda x:pd.Series([x.iloc[0]['r2_predict'],r2_score(x['target_qtty'],x['prediction'])],index=['train_r2','valid_2_r2']))\
                             .reset_index()

result_df4_pd = result_df3_pd.groupby('item_sku_id').apply(lambda x:x.sort_values('valid_2_r2',ascending=False).head(1)).reset_index(drop=True)
result_df4 = spark.createDataFrame(result_df4_pd)
result_df4.cache()
result_df4.count()

1950

In [8]:
# 找历史最高价、最低价、历史最高销量
# sale_qtty可以改成target_qtty
result_df5 = result_df2.join(result_df4.select(['item_sku_id','model_type']),['item_sku_id','model_type'],'inner')

history_price = df_2.filter(F.col('test_flag')==0).groupby('item_sku_id')\
                    .agg(F.min('target_price').alias('min_price'),F.max('target_price').alias('max_price'),F.max('sale_qtty').alias('max_qtty'))

# 加入训练集最后一天的特征值
last_day_feature = df_2.filter(F.col('test_flag')==0).withColumn('last_day_flag',F.row_number().over(Window.partitionBy('item_sku_id').orderBy(F.col('dt').desc())))
last_day_feature = last_day_feature.filter(F.col('last_day_flag')==1).select(list(set(['item_sku_id']+X_SCHEMA_SKU)-set(['target_price'])))

second_df = result_df5.join(history_price,['item_sku_id'],'inner')\
                      .join(last_day_feature,['item_sku_id'],'inner')\
                      .select(['item_sku_id','dt','valid_flag','model','model_type','target_qtty','prediction','min_price','max_price','max_qtty']+X_SCHEMA_SKU)
# second_df.cache()
# second_df.count()

second_df.write.mode('overwrite').format('orc').saveAsTable('dev.dev_xianzhi_model_data_6_zou')
spark.sql('''ALTER TABLE dev.dev_xianzhi_model_data_6_zou set TBLPROPERTIES ('author'='zoushuhan') ''')

DataFrame[]

In [3]:
X_SCHEMA_SKU = ['target_price', 'decomposedtrend','cid3trend','target_days_flag']
second_df = spark.sql('''select * from dev.dev_xianzhi_model_data_6_zou ''')

second_df.cache()
second_df.count()

265866

In [4]:
import pickle
import codecs
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble  import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor

SCHEMA_OUTPUT_SKU2 = StructType([
    StructField("item_sku_id", sql_type.StringType()),
    StructField("dt", sql_type.StringType()),
    StructField("valid_flag", sql_type.FloatType()),
    StructField("model_type", sql_type.StringType()),
    StructField("model", sql_type.StringType()),
    StructField("huber_model", sql_type.StringType()),
    StructField("max_qtty", sql_type.FloatType()),
    StructField("target_qtty", sql_type.FloatType()),
    StructField("target_price", sql_type.FloatType()),
    StructField("prediction", sql_type.FloatType()),
    StructField("second_prediction", sql_type.FloatType())
])


def format_result_sku2(row):
    return (
        str(row['item_sku_id']),
        str(row['dt']),
        float(row['valid_flag']),
        str(row['model_type']),
        str(row['model']),
        str(row['huber_model']),
        float(row['max_qtty']),
        float(row['target_qtty']),
        float(row['target_price']),
        float(row['prediction']),
        float(row['second_prediction'])
    )

def yichang(ddf):
    ddf = ddf.reset_index()
    if list(ddf.sort_values('target_price')['index']) != list(ddf.sort_values('target_qtty',ascending=False)['index']):
        return 1
    elif list(ddf.sort_values('target_price')['index']) != list(ddf.sort_values('prediction',ascending=False)['index']):
        return 1
    else:
        return 0


def smooth(row):    
    df = pd.DataFrame(list(row[1]),columns=['item_sku_id','dt','valid_flag','model','model_type','target_qtty','prediction','min_price','max_price','max_qtty']+X_SCHEMA_SKU)
    sku_model = pickle.loads(codecs.decode(df.iloc[0]['model'].encode(), 'base64'))
    # 做最大最小间隔，带入原模型，得到预测值
    xuni_feature = pd.DataFrame({'target_price':list(range(int(df.iloc[0]['min_price']),int(df.iloc[0]['max_price']+3)))})
    # 用训练集最后一天填入其他特征的值
    for fea in list(set(X_SCHEMA_SKU)-set(['target_price'])):
        xuni_feature[fea] = df.iloc[0][fea]
    xuni_y = np.exp(sku_model.predict(xuni_feature[X_SCHEMA_SKU]))-1
    xuni_df = pd.concat([xuni_feature,pd.DataFrame({'xuni_y':xuni_y})],axis=1)
#     xuni_df['target_price_2'] = xuni_df['target_price']*xuni_df['target_price']
    # 建立线性模型
    sku_lm_model = HuberRegressor()
    sku_model_lm = sku_lm_model.fit(xuni_df[['target_price']],xuni_df[['xuni_y']])
    # 带入test的X
#     df['target_price_2'] = df['target_price']*df['target_price']   
    xuni_ls = sku_model_lm.predict(df[['target_price']])
    huber_m = codecs.encode(pickle.dumps(sku_model_lm), "base64").decode()
#     result = pd.concat([df[['item_sku_id','dt','valid_flag','model_type','target_qtty','prediction']+X_SCHEMA_SKU],pd.DataFrame({'second_prediction':list(map(lambda x:x[0],xuni_ls))})],axis=1)
    
    yichang_dt = df.groupby('dt').apply(lambda x:yichang(x)).reset_index().rename(columns={0:'dt_flag'})
    yichang_dt = list(yichang_dt.loc[yichang_dt['dt_flag']==1,'dt'])
    
    result = pd.concat([df[['item_sku_id','dt','valid_flag','model','model_type','max_qtty','target_qtty','prediction']+X_SCHEMA_SKU],pd.DataFrame({'second_prediction':list(xuni_ls)})],axis=1)
    result.loc[result['dt'].isin(yichang_dt)==False,'second_prediction'] = result.loc[result['dt'].isin(yichang_dt)==False,'prediction']
    
    result['huber_model'] = huber_m 
    result = result[['item_sku_id','dt','valid_flag','model_type','model','huber_model','max_qtty','target_qtty']+X_SCHEMA_SKU+['prediction','second_prediction']]
    
    # 销量要小于等于历史最大销量，且大于等于0
    result.loc[result['second_prediction']>df.iloc[0]['max_qtty'],'second_prediction'] = df.iloc[0]['max_qtty']
    result.loc[result['second_prediction']<0,'second_prediction'] = 0
    
    return result.to_dict(orient= 'records')

In [5]:
# 随机森林做huber
second_result = second_df.filter(F.col('model_type')=='rf').rdd.map(lambda row: ((row['item_sku_id']), row)).groupByKey()\
    .flatMap(lambda row : smooth(row))

second_result2 = spark.createDataFrame(second_result.map(format_result_sku2), schema=SCHEMA_OUTPUT_SKU2)
second_result2.cache()
second_result2.count()

71804

In [6]:
# 随机森林和其他的再union到一起
second_result3 = second_result2.select('item_sku_id','dt','valid_flag','model_type','model','huber_model','max_qtty','target_qtty','target_price','prediction','second_prediction')\
                               .union(second_df.filter(F.col('model_type')!='rf').withColumn('huber_model',F.lit(None))\
                                               .select('item_sku_id','dt','valid_flag','model_type','model','huber_model','max_qtty','target_qtty','target_price','prediction')\
                                               .withColumn('second_prediction',F.col('prediction')))



second_result3.cache()
second_result3.count()

265866

In [7]:
# 加入计算mape的列
second_result3 = second_result3.withColumn('percent',F.abs(F.col('target_qtty')-F.col('prediction'))/F.when(F.col('target_qtty')==0,F.lit(1)).otherwise(F.col('target_qtty')))\
                               .withColumn('second_percent',F.abs(F.col('target_qtty')-F.col('second_prediction'))/F.when(F.col('target_qtty')==0,F.lit(1)).otherwise(F.col('target_qtty')))


In [8]:
second_result3.write.mode('overwrite').format('orc').saveAsTable('dev.dev_xianzhi_model_data_6_result_zou')
spark.sql('''ALTER TABLE dev.dev_xianzhi_model_data_6_result_zou set TBLPROPERTIES ('author'='zoushuhan') ''')

DataFrame[]

In [9]:
second_result3 = spark.sql('''select * from dev.dev_xianzhi_model_data_6_result_zou ''')
second_result3.cache()
second_result3.count()

265866

In [10]:
second_result3.filter(F.col('valid_flag')==2).agg(F.max('dt')).show()

+----------+
|   max(dt)|
+----------+
|2019-07-11|
+----------+



In [11]:
second_result3.agg(F.max('dt')).show()

+----------+
|   max(dt)|
+----------+
|2019-08-25|
+----------+



In [12]:
# sum(result_table2['percent'])/len(result_table2)
second_result3.agg(F.mean('second_percent')).show()

+-------------------+
|avg(second_percent)|
+-------------------+
| 3.5730705304489088|
+-------------------+



In [13]:
# sum(result_table2.loc[result_table2['valid_flag']==2,'percent'])/len(result_table2.loc[result_table2['valid_flag']==2))
second_result3.filter(F.col('valid_flag')==2).agg(F.mean('second_percent')).show()

+-------------------+
|avg(second_percent)|
+-------------------+
| 2.9055066070557585|
+-------------------+



In [14]:
# sum(result_table2.loc[(result_table2['valid_flag']==2)&(result_table2['target_qtty']>=10),'percent'])/len(result_table2.loc[(result_table2['valid_flag']==2)&(result_table2['target_qtty']>=10)])
second_result3.filter(F.col('valid_flag')==2).filter(F.col('target_qtty')>=10).agg(F.count('second_percent'),F.mean('second_percent')).show()

+---------------------+-------------------+
|count(second_percent)|avg(second_percent)|
+---------------------+-------------------+
|                 4880| 0.8720085390860289|
+---------------------+-------------------+



In [15]:
second_result3.filter(F.col('valid_flag')==2).filter(F.col('target_qtty')>=10)\
    .filter(F.col('item_sku_id').isin(['6258780','100003741320','407901','2918980','6083253'])==False).agg(F.count('second_percent'),F.mean('second_percent')).show()

+---------------------+-------------------+
|count(second_percent)|avg(second_percent)|
+---------------------+-------------------+
|                 4843| 0.7896462753338485|
+---------------------+-------------------+



In [16]:
# 按sku算mape再平均
second_result3.filter(F.col('valid_flag')==2).filter(F.col('target_qtty')>=10)\
    .filter(F.col('item_sku_id').isin(['6258780','100003741320','407901','2918980','6083253'])==False).groupby('item_sku_id').agg(F.count('second_percent'),F.mean('second_percent').alias('mape')).agg(F.mean('mape')).show()

+------------------+
|         avg(mape)|
+------------------+
|0.8316104815208014|
+------------------+



In [17]:
# sum(abs(result_table2.loc[(result_table2['valid_flag']==2)&(result_table2['target_qtty']>=10),'target_qtty']-
#         result_table2.loc[(result_table2['valid_flag']==2)&(result_table2['target_qtty']>=10),'prediction']))/sum(result_table2.loc[(result_table2['valid_flag']==2)&(result_table2['target_qtty']>=10),'target_qtty'])
wmape = second_result3.withColumn('cha',F.abs(F.col('target_qtty')-F.col('second_prediction')))
sum_qtty = wmape.filter(F.col('valid_flag')==2).filter(F.col('target_qtty')>=10)\
                .agg(F.sum('target_qtty')).collect()[0][0]

wmape.filter(F.col('valid_flag')==2).filter(F.col('target_qtty')>=10)\
     .agg((F.sum('cha')/sum_qtty).alias('wmape')).show()

# second = 0.45

+------------------+
|             wmape|
+------------------+
|0.5273395108177168|
+------------------+



In [18]:
# sum(abs(result_table2.loc[(result_table2['valid_flag']==2)&(result_table2['target_qtty']>=10),'target_qtty']-
#         result_table2.loc[(result_table2['valid_flag']==2)&(result_table2['target_qtty']>=10),'prediction']))/sum(result_table2.loc[(result_table2['valid_flag']==2)&(result_table2['target_qtty']>=10),'target_qtty'])
wmape = second_result3.withColumn('cha',F.abs(F.col('target_qtty')-F.col('second_prediction')))
sum_qtty = wmape.filter(F.col('valid_flag')==2).filter(F.col('target_qtty')>=10)\
                .filter(F.col('item_sku_id').isin(['6258780','100003741320','407901','2918980','6083253'])==False)\
                .agg(F.sum('target_qtty')).collect()[0][0]

wmape.filter(F.col('valid_flag')==2).filter(F.col('target_qtty')>=10)\
     .filter(F.col('item_sku_id').isin(['6258780','100003741320','407901','2918980','6083253'])==False)\
     .agg((F.sum('cha')/sum_qtty).alias('wmape')).show()

# second = 0.45

+------------------+
|             wmape|
+------------------+
|0.5163970197688419|
+------------------+



In [19]:
second_result3.filter(F.col('valid_flag')==2).filter(F.col('target_qtty')>=10).filter(F.col('item_sku_id').isin(['6258780','100003741320','407901','2918980','6083253'])==False)\
              .groupby('model_type').agg(F.count('percent')).show()

+----------+--------------+
|model_type|count(percent)|
+----------+--------------+
|        rf|          1536|
|        lm|           633|
|   xgboost|          1788|
|   prophet|           886|
+----------+--------------+



6258780 prophet 有异常，不过sku其中有一段下柜的跳跃  
100003741320  训练集中本身都是39-40元卖200-400个，但测试集中41元就只能卖10-20个  
407901  测试集中很多天都是价格高销量高，反向数据多，需要剔除  
2918980 sku本身波动太大，同样的价格前后销量差距很大